In [1]:
import os
import sys
import json
import pandas as pd
import math

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
   raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
sc = spark.sparkContext

In [4]:
k=50
z=3.291

In [5]:
splited = sc.textFile("/labs/lab06data/ml-100k/u.data").map(lambda x: x.split("\t")).cache()

In [6]:
film_count = splited.map(lambda x: (int(x[1]), 1)).reduceByKey(lambda a,b: a+b).sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0])

In [7]:
top10_rates = film_count.take(10)

In [8]:
film_avg = splited.map(lambda x: (x[1], int(x[2]))).groupByKey().map(lambda x: (x[0], sum(x[1])/len(x[1])))\
.join(sc.textFile("/labs/lab06data/ml-100k/u.item").map(lambda x: x.split("|")))\
.map(lambda x: (x[0], x[1][0], x[1][1])).sortBy(lambda x: (-x[1],x[2])).map(lambda x: x[0])

In [9]:
top10_average = film_avg.take(10)

In [10]:
global_average = splited.map(lambda x: ('temp',int(x[2]))).groupByKey().map(lambda x: (x[0], sum(x[1])/len(x[1]))).map(lambda x: x[1]).collect()[0]

In [11]:
top10_rating = splited.map(lambda x: (int(x[1]), int(x[2]))).groupByKey().map(lambda x: (x[0], (sum(x[1])+k*global_average)/(len(x[1])+k)))\
.sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0]).take(10)

In [12]:
def calc_lower_board(positive_part, size):
     return (1/(1+z**2/size))*(positive_part+z**2/(2*size)-z*math.sqrt(1/size*positive_part*(1-positive_part)+z**2/(4*size**2)))

In [13]:
top10_lower = splited.map(lambda x: (int(x[1]), 1 if int(x[2])>3 else 0)).groupByKey().map(lambda x: (x[0], len(x[1]), sum(x[1])/len(x[1])))\
.map(lambda x: (x[0], calc_lower_board(x[2], x[1]))).sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0]).take(10)

In [14]:
top10_lower

[64, 98, 318, 479, 50, 483, 603, 427, 357, 12]

In [15]:
res = {"top10_rates":top10_rates, "top10_average":top10_average, "top10_rating":top10_rating, "top10_lower":top10_lower}

In [16]:
with open("lab06s.json", 'w') as file:
    json.dump(res, file)

In [17]:
sc.stop()